In [ ]:
# !pip install 'sagemaker[local]' sagemaker --upgrade 

In [5]:
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer, json_serializer


sess = sagemaker.Session()
role = 'arn:aws:iam::514965996716:role/service-role/AmazonSageMaker-ExecutionRole-20181229T135829'# get_execution_role()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

In [6]:
training_data = 's3://sagemaker-eu-west-1-514965996716/data/plant-disease-dataset/train'
test_data = 's3://sagemaker-eu-west-1-514965996716/data/plant-disease-dataset/valid'

In [10]:
image = '514965996716.dkr.ecr.eu-west-1.amazonaws.com/autogluon-image-classification'

sm_model = Estimator(
    image_name=image,
    role=role, 
    train_instance_count=1, 
    train_instance_type='ml.p2.xlarge',
    hyperparameters={'epochs': 2, 'ngpus-per-trial': 1},
    sagemaker_session=sess,
    train_use_spot_instances=True,
    train_max_wait=7200,
    train_volume_size=100,
    train_max_run=7200,
)

In [11]:
sm_model.fit({'training': training_data, 'testing': test_data})

2020-03-07 11:28:16 Starting - Starting the training job...
2020-03-07 11:28:18 Starting - Launching requested ML instances...
2020-03-07 11:29:16 Starting - Preparing the instances for training............
2020-03-07 11:30:56 Downloading - Downloading input data..............................................................................
2020-03-07 11:44:38 Training - Downloading the training image...../usr/local/lib/python3.7/site-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))
Images Classes: ['Citrus Mosaic 1', 'Cherry_(including_sour)___Powdery_mildew', 'Black Pepper Phytophthora Foot Rot 1', 'Citrus Alternaria Brown Spot 1', 'Dry Bean Phytotoxicity 1', 'Soybean Cercospora Blight Cropped', 'Melon Bacterial Fruit Blotch 1', 'Orange___Haunglongbing_(Citrus_greening)', 'Corn Physoderma Brown Spot Cropped', 'Sugarcane Ru


2020-03-07 11:45:50 Uploading - Uploading generated training model
2020-03-07 11:46:36 Failed - Training job failed


UnexpectedStatusException: Error for Training job autogluon-image-classification-2020-03-07-11-28-15-891: Failed. Reason: AlgorithmError: Exception during training: '>' not supported between instances of 'str' and 'int'
Traceback (most recent call last):
  File "/opt/program/train", line 43, in train
    output_directory=model_dir
  File "/usr/local/lib/python3.7/site-packages/autogluon/task/image_classification/image_classification.py", line 243, in fit
    ngpus_per_trial = get_gpu_count() if ngpus_per_trial > get_gpu_count() else ngpus_per_trial
TypeError: '>' not supported between instances of 'str' and 'int'


In [ ]:
predictor = sm_model.deploy(1, 'ml.m4.xlarge', serializer=json_serializer)

In [ ]:
import pandas as pd
import json
import io

In [ ]:
df = pd.read_csv('s3://datafy-data-lake-public-artifacts/Inc/test.csv')

In [ ]:
data = df.head()[['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']]

In [ ]:
record = data.head(1).T.to_dict()[0]

payload = json.dumps(record)

In [ ]:
d = predictor.predict(payload)

In [ ]:
predictor.delete_endpoint()